In [ ]:
#Project Title:- 

In [ ]:
#The following code will scrap the IDMB site to get user reviews for movies under genre Sci-fi

In [1]:
#Use the webdriver

# set webdriver path (pay attention: it is an absolute path)
# webdriver installaion
# selenium version 3.141.0, chorme version 111
driver_path = '/Users/amy/Downloads/chromedriver_mac_arm64 1/chromedriver'

# IMDb movie number
movie_code = 'tt6723592'  # TENET
# how many comment pages should be displayed (to control the number of comments)
pages = 5



In [2]:
#automatically collect all reviews


import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support import expected_conditions as EC

# setting webdriver
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
options.add_argument("--disable-extensions")
options.add_argument("--disable-notifications")



# initial webdriver

#Ladies, I find that I only can use the absolute path of the webdriver --Wentao
# driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver = webdriver.Chrome(executable_path=driver_path, options=options)

# Open the IMDb movie review page
driver.get(f'https://www.imdb.com/title/{movie_code}/reviews?ref_=tt_urv')
# driver.get('https://www.imdb.com/title/tt11219254/reviews?ref_=tt_urv')

# Wait for the page to load
wait = WebDriverWait(driver, 10)
wait.until(EC.presence_of_element_located((By.ID, 'load-more-trigger')))

# Select 'Review Date' from the 'sort by' drop-down box
select = Select(driver.find_element_by_xpath("//select[@name='sort']"))
select.select_by_visible_text("Review Date")

# Wait for the page to load
wait = WebDriverWait(driver, 10)
wait.until(EC.presence_of_element_located((By.ID, 'load-more-trigger')))

# Click the "Load More" button 5 times and wait for more comments to load
for i in range(pages):
    load_more_button = driver.find_element_by_id('load-more-trigger')
    load_more_button.click()
    wait.until(EC.invisibility_of_element_located((By.CSS_SELECTOR, '.ipl-load-more__load-indicator')))

# Get the content of all the comments and analyze them
reviews = driver.find_elements_by_class_name('review-container')
for item_content in reviews:
    spoiler_warning = item_content.find_elements_by_class_name("spoiler-warning")
    if spoiler_warning:
#         print("Spoiler warning class is present in lister-item-content class")
        spoiler_link = item_content.find_element_by_class_name("ipl-expander")
        spoiler_link.click()

# Get the movie's name
movie_name = driver.find_element_by_tag_name('h3').text.strip()
# print(movie_name)

data = []
for i, review in enumerate(reviews):
    # get the rating
    try:
        rating = review.find_element_by_class_name('rating-other-user-rating').text.strip()
    except:
        rating = ''
    # get the username
    try:
        username = review.find_element_by_class_name('display-name-link').text.strip()
    except:
        username = ''
    # get the review's title
    try:
        title = review.find_element_by_class_name('title').text.strip()
    except:
        title = ''
    # get the content of a review
    try:
        content = review.find_element_by_class_name("text").text.strip()
    except:
        content = ''
    # get the review's date
    try:
        time = review.find_element_by_class_name('review-date').text.strip()
    except:
        time = ''
    # add them into the data
    data.append([title, username,rating, time, content])

# save data into CSV file
with open(f'{movie_name}_{movie_code}_reviews.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['Title', 'User', 'StarsRating', 'Date', 'ReviewText'])
    writer.writerows(data)

# close webdriver
driver.quit()


In [1]:
# import requests
# import csv
# from bs4 import BeautifulSoup
# import time

# # URL of the movie's reviews page on IMDB sorted by review date
# url = 'https://www.imdb.com/title/tt6723592/reviews?sort=submissionDate&dir=desc&ratingFilter=0'

# # Send a request to the page and get its content
# response = requests.get(url)
# html_content = response.content

# # Parse the HTML content using BeautifulSoup
# soup = BeautifulSoup(html_content, 'html.parser')

# # Find the movie title
# movie_title = soup.find('h1').text.strip()

# # # Find the "Load More" button
# # load_more = soup.find('button', class_='ipl-load-more__button')

# # # Loop until all reviews are displayed
# # while load_more is not None:
# #     # Get the AJAX endpoint URL from the "Load More" button
# #     endpoint = load_more.get('data-load-more-href')
# #     if not endpoint:
# #         break

# #     # Send an AJAX request to get more reviews
# #     response = requests.get(f'https://www.imdb.com{endpoint}')
# #     data = response.json()

# #     # Append the new reviews to the HTML content
# #     html_content += data['load_more_html'].encode()

# #     # Wait a few seconds before making the next request
# #     time.sleep(2)

# #     # Update the BeautifulSoup object with the new HTML content
# #     soup = BeautifulSoup(html_content, 'html.parser')

# #     # Find the new "Load More" button
# #     load_more = soup.find('div', class_='load-more-data')

# # Find all the reviews on the page
# reviews = soup.find_all('div', class_='imdb-user-review')

# # Open a CSV file to write the results
# with open(f'{movie_title}_reviews.csv', mode='w', newline='', encoding='utf-8') as file:
#     writer = csv.writer(file)

#     # Write the header row of the CSV file
#     writer.writerow(['Title', 'User', 'Stars', 'Date', 'Content'])

#     # Loop through each review and write its data to the CSV file
#     for review in reviews:
#         title = review.find('a', class_='title').text
#         user = review.find('span', class_='display-name-link').text
#         stars = review.find('span', class_='rating-other-user-rating').find(
#             'span').text if review.find('span', class_='rating-other-user-rating') else 'N/A'
#         date = review.find('span', class_='review-date').text
#         content = review.find(
#             'div', class_='text show-more__control').text.strip()
#         writer.writerow([title, user, stars, date, content])


In [ ]:
#The following code will clean the data collected to remove any non-alphanumeric characters and build a dataframe

In [8]:
import csv
import re
import pandas as pd

list_of_review_data = []
with open('User Reviews_reviews.csv',newline='') as csvfile:
    data = csv.reader(csvfile, delimiter=',')
    for row in data:
        list_of_review_data.append(row)
list_of_review_data = list_of_review_data[1:]
for i in range(len(list_of_review_data)):
    list_of_review_data[i][0] = re.sub(r'[!@#$%^()&*+=,\\\n]', '', list_of_review_data[i][0])
    list_of_review_data[i][4] = re.sub(r'[!@#$%^()&*+=,\\\n]', '', list_of_review_data[i][4])

df = pd.DataFrame(list_of_review_data, columns =['Title', 'User', 'StarsRating', 'Date', 'ReviewText'])
df

,Title,User,StarsRating,Date,ReviewText
0,Not for all but for some,ZinedineElbaroni,8,13 March 2023,Tenet is a cinematic masterpiece that takes yo...
1,Damned good try,xyqkp,7,10 March 2023,Putting aside the ham-handed effort to explain...
2,bloated pretentious,Chris_Ego,2,9 March 2023,I have to admit crashing a real plane with pra...
3,Tall Drink of Water,sure_man,4,8 March 2023,Although this was victimized by COVID even aft...
4,A Mind-Bending and Spectacular Spy Thriller,FilmFanatic2023,8,4 March 2023,Tenet is a sci-fi action film directed by Chri...
5,Another masterpiece crafted by the one and on...,Nicholas_HY,10,20 February 2023,My only regret was not watching this in the th...
6,Another amazing film by Christopher Nolan,Reviewosaurus,9,19 February 2023,After a near-death experience during a CIA ope...
7,Worth to watch,giedriavas,8,17 February 2023,A film will discover both fans and those who w...
8,A movie that bit more than it could chew,dargosmunteanu,8,17 February 2023,The concept behind Tenet is a very interesting...
9,Phantasy of a genius director but a plain movie,furkankilickap,8,14 February 2023,Christopher Nolan's filmography has in my view...


In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer
ps = PorterStemmer()

# remove the stop words
stop_words = set(stopwords.words('english'))
def remove_stopwords(tokens):
    return [token for token in tokens if token.lower() not in stop_words]


# tokenize the reviews
df['sentences'] = df['ReviewText'].apply(lambda x: sent_tokenize(x))
df['tokens'] = df['sentences'].apply(lambda x: [remove_stopwords(word_tokenize(sentence)) for sentence in x])
df = df.drop('sentences', axis=1)


# pos-tagging the reviews
df['pos_tags'] = df['tokens'].apply(lambda x: [nltk.pos_tag(sentence) for sentence in x])


# stemming the reviews
df['stemmed'] = df['pos_tags'].apply(lambda x: [[ps.stem(word) for word, tag in sentence] for sentence in x])

df